In [17]:
import datetime as dt
from datetime import timezone, timedelta
import os
import platform
import sys

In [2]:
# Melhorar com:
# Separar as funções existentes: Saque, Depósito e Extrato
    ## A função saque deve receber os argumentos apenas por nome (saldo=saldo, valor=valor... etc etc) e retornar algo
    ## A função Depósito deve receber os argumentos apenas por posição
    # A função extrato pode receber por ambas as formas
        # parametro Saldo: posicional
        # parametro extrato: nomeado
# Criar cadastrar usuário e cadastrar conta bancária
    # Usuário armezado em uma lista contendo: nome, dt_nascimento, cpf e endereço (é um string com: logradouro, numero, bairro, cidade, estado(sigla)) - CPF tem que filtrar para ter apenas numeros (mesmo sendo string)
        # Não posso cadastrar 2 cpf iguais
    # Conta armazenado em uma lista contendo: agência, numero da conta, usuário. 
        #Numero da conta é sequencial começando em 1. 
        # Numero da agencia é fixo 0001
        # Usuários podem ter várias contas, mas cada conta só pode ser de 1 usuário

In [20]:
def menu():
    menu = '''

    [d] Depositar  
    [s] Sacar      
    [e] Extrato    
    [q] Sair       
    
    => '''
    return input(menu).lower()

def clear_terminal():
    """Limpa o terminal, independentemente do sistema operacional ou se está no Jupyter Notebook."""
    try:
        # Verifica se está rodando no Jupyter Notebook
        if 'ipykernel' in sys.modules:
            from IPython.display import clear_output
            clear_output(wait=True)
        else:
            os.system('cls' if platform.system() == 'Windows' else 'clear')
    except Exception as e:
        print(f"Erro ao limpar o terminal: {e}")

def current_datetime():  
    return dt.datetime.now(timezone(timedelta(hours=-3)))
    
def format_complete_date(date):  
    return date.strftime("%d/%m/%Y %H:%M:%S")

def format_simple_date(date):  
    return date.strftime("%d/%m/%Y")    

def transactions_number(transactions, date, limit): 
    return transactions.count(date) < limit

def transactions_daily(transactions, date):
    return transactions.count(date) 

def withdraw(*, balance, statement, current_date):
    if(transactions_number(transactions, format_simple_date(current_datetime()), TRANSACTIONS_LIMIT)):
        try:
            withdraw_value = float(input('Digite o valor que deseja sacar:'))  # valor_saque -> withdraw_value
            clear_terminal()
            if (withdraw_value <= 0):
                print("Valor Inválido, tente novamente.")
            elif (withdraw_value <= DAILY_LIMIT and balance >= withdraw_value):
                transactions.append(current_date)
                balance -= float(withdraw_value)
                print(f'Saque de {withdraw_value} realizado com sucesso.')
                return withdraw_value, balance
            elif withdraw_value > balance:
                print('Saldo insuficiente.')
            else:
                print("Operação Inválida, confira seu extrato.")
        except ValueError:
            print("Erro: Você deve inserir um valor numérico válido para o saque.")
            return None, balance
    else:
        print("Limite de Operações Excedido. Confira seu extrato")
        return None, balance
def deposit (balance, current_date, /):
    if(transactions_number(transactions, format_simple_date(current_datetime()), TRANSACTIONS_LIMIT)):
        try:
            deposit_value = float(input('Digite o valor que deseja depositar:')) 
            clear_terminal()
            if (deposit_value <= 0):
                print("Valor Inválido, tente novamente.")
            else:
                balance += float(deposit_value)
                transactions.append(current_date)
                print(f"Depósito de R${deposit_value:.2f} realizado com sucesso.")
                return deposit_value, balance
        except ValueError:
            print("Erro: Você deve inserir um valor numérico válido para o depósito.")
            return None, balance
    else:
        print("Limite de Operações Excedido. Confira seu extrato")
        return None, balance

def account_statement (balance, /, *, statement):
    print("\n=============== EXTRATO =================")
    print('Não foram realizadas movimentações' if not statement else statement)
    print(f'Saldo: R${balance:.2f}')
    print(f'Seu limite diário de saque é de: R${DAILY_LIMIT:.2f}')
    print(f'Você já realizou {transactions_daily(transactions, format_simple_date(current_datetime()))} transações hoje.')
    print("==========================================")

DAILY_LIMIT = 500
TRANSACTIONS_LIMIT = 10

balance = float(0)  # saldo -> balance
statement = ""  # extrato -> statement
transactions = []  

while True:
    clear_terminal()
    option = menu()
    current_date = format_simple_date(current_datetime()) 
    
    if option == 'd':
        clear_terminal()
        deposit_value, balance = deposit(balance, current_date)
        if deposit_value is not None:
            statement += f'Depósito de {deposit_value} realizado em {format_complete_date(current_datetime())}\n'
        
    elif option == 's':
        clear_terminal()
        withdraw_value, balance = withdraw(balance=balance, statement=statement, current_date=current_date)
        if withdraw_value is not None:
            statement += f'Saque de {withdraw_value} realizado em {format_complete_date(current_datetime())}\n'
        
    elif option == 'e':
        clear_terminal()
        account_statement(balance, statement=statement)
        
    elif option == 'q':
        clear_terminal()
        break
    else:
        print("Operação inválida. Selecione um menu.")



    [d] Depositar  
    [s] Sacar      
    [e] Extrato    
    [q] Sair       
    
    =>  q
